In [4]:
%load_ext autoreload
%autoreload 2
from main import init_model, training_loop

tokenizer, model = init_model()

2025-04-24 20:27:53.857 | INFO     | main:init_model:34 - Loading model Qwen/Qwen2.5-3B-Instruct


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.38s/it]
2025-04-24 20:28:03.127 | INFO     | main:init_model:37 - Model loaded.
2025-04-24 20:28:03.128 | INFO     | main:init_model:39 - Using device mps


In [12]:
from dataset import create_connections_datasets
from main import get_available_device
train_dataset, _ = create_connections_datasets()
device = get_available_device()
training_loop(model, tokenizer, train_dataset, device)

connections_prompts.jsonl
                                                 groups  \
4896  [{'words': ['baltic', 'black', 'philippine', '...   
4782  [{'words': ['even', 'level', 'stable', 'steady...   
1496  [{'words': ['glass', 'old', 'signs', 'split'],...   
1957  [{'words': ['guard', 'mind', 'tend', 'watch'],...   
9171  [{'words': ['kind', 'sort', 'type', 'variety']...   
...                                                 ...   
5734  [{'words': ['aioli', 'barbecue', 'marinara', '...   
5191  [{'words': ['catfish', 'cribs', 'jackass', 'st...   
5390  [{'words': ['hop', 'jump', 'leap', 'spring'], ...   
860   [{'words': ['cone', 'cube', 'pyramid', 'sphere...   
7270  [{'words': ['aces', 'keen', 'neato', 'nifty'],...   

                                                  words  
4896  [baltic, black, philippine, red, bear, harbor,...  
4782  [even, level, stable, steady, creamsicle, jour...  
1496  [glass, old, signs, split, airplane, mother, o...  
1957  [guard, mind, tend, watch, 

TypeError: sequence item 0: expected str instance, list found